In [1]:
from taxcalc import *
from bokeh.io import show, output_notebook
from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.models import HoverTool, ColumnDataSource, NumeralTickFormatter
from collections import OrderedDict
import copy
import pandas as pd
from notebookfunctions import distribution, index_list, percentile
output_notebook()

Loading BokehJS ...

In [2]:
# Data from IRS-SOI Tax Stats. Used for comparison
soi_stats = pd.read_csv('soi_stats.csv', index_col=0)  # Equivalent to tax-calc diagnostic table
soi_income = pd.read_csv('soi_income_stats.csv')  # Distribution of income items
soi_deductions = pd.read_csv('soi_deductions.csv', index_col=0)  # Itemized deductions
soi_deductions['index'] = soi_deductions.index

In [3]:
# Read in new PUF and associated data
gf = pd.read_csv('../taxdata/stage1/growfactors.csv')
wt = pd.read_csv('../taxdata/puf_stage2/puf_weights.csv')
puf = pd.read_csv('../taxdata/puf_data/puf.csv')
adj = pd.read_csv('../taxdata/puf_stage3/puf_ratios.csv', index_col=0)
adj = adj.transpose()

In [4]:
# base calculator
# temporary growth rates
grow_temp = Growfactors('taxcalc/growfactors_temp.csv')
recs = Records(gfactors=grow_temp)
pol = Policy()
calc = Calculator(records=recs, policy=pol)
calc.advance_to_year(2014)
calc.calc_all()

You loaded data for 2009.
Tax-Calculator startup automatically extrapolated your data to 2013.


In [5]:
# new calculator
growf = Growfactors('../taxdata/stage1/growfactors.csv')
rec = Records(puf, gfactors=growf,
              weights=wt, adjust_ratios=adj,
              start_year=2011)
pol = Policy(gfactors=growf, start_year=2011, num_years=17)
consump = Consumption(start_year=2011, num_years=17)
behave = Behavior(start_year=2011, num_years=17)
calc_new = Calculator(records=rec, policy=pol, consumption=consump, behavior=behave)
calc_new.advance_to_year(2014)
calc_new.calc_all()

You loaded data for 2011.
Tax-Calculator startup automatically extrapolated your data to 2011.


In [6]:
# list of variables used throughout notebook
var_list = ['e00200', 'e00300', 'e00600', 'e00650', 'e00900', 'c00100',
            's006', 'e17500', 'c04470', 'e18400', 'e18500', 'e19200', 'e19800', 'e20100',
            'e20400', 'g20500', 'iitax', 'payrolltax', 'combined']
# create data frames for both calculators
calc_df = calc.dataframe(var_list)
new_df = calc_new.dataframe(var_list)

# Distribution Tables

In [7]:
curr_table, _ = calc.distribution_tables(None)
print ('Current PUF')
curr_table

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


Current PUF


,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,221,793","-87,307,405,375","11,146,637","96,407,359,841","6,087","264,420,507","66,734,350,903","837,996,365","80,929,000","-93,697,347,981",...,0,"80,929,000","6,971,601","81,400,811","2,526,565,011","-2,371,206,801","5,813,976,106","3,442,769,306","-83,056,124,135","-86,498,893,441"
1,"16,222,840","97,696,000,447","13,632,755","114,464,028,623","54,252","427,192,044","79,940,587,312","4,098,599,114","366,554,922","97,335,286,959",...,"7,933,783","374,488,705","7,163,854","67,319,871","12,374,838,446","-11,940,193,724","13,166,598,731","1,226,405,007","136,744,925,045","135,518,520,038"
2,"16,221,490","163,468,877,532","13,230,158","123,397,911,082","291,268","3,127,562,614","105,324,141,284","23,615,280,936","2,317,284,610","161,287,246,966",...,"28,785,086","2,346,069,695","313,479,968","95,718,940","26,344,313,834","-24,216,005,166","22,653,526,956","-1,562,478,210","243,042,824,393","244,605,302,603"
3,"16,223,700","244,271,226,308","14,193,383","126,510,866,481","665,097","8,675,779,270","110,406,827,660","64,167,133,999","6,728,920,955","238,716,968,823",...,"29,752,265","6,758,673,220","1,317,901,363","190,009,074","23,461,455,686","-17,830,674,754","32,516,776,026","14,686,101,272","355,123,480,068","340,437,378,795"
4,"16,222,475","362,531,321,371","14,165,525","128,715,334,378","1,497,871","20,700,893,573","116,829,492,149","139,146,548,563","16,078,090,151","349,707,249,795",...,"44,135,661","16,122,225,812","3,488,450,770","262,896,068","17,584,563,210","-4,687,892,100","46,562,008,652","41,874,116,551","491,474,693,643","449,600,577,091"
5,"16,221,683","524,486,317,326","13,178,967","124,460,625,287","2,882,627","43,092,458,811","118,837,408,328","265,330,113,463","32,309,372,364","498,816,566,335",...,"40,562,634","32,349,934,997","5,828,747,564","426,620,413","8,995,351,586","17,952,456,261","63,669,887,689","81,622,343,950","660,912,932,518","579,290,588,568"
6,"16,222,444","748,546,418,528","11,388,577","115,287,352,674","4,803,785","80,625,803,708","125,281,678,668","437,438,543,361","57,283,794,507","700,594,959,360",...,"38,919,584","57,322,714,090","8,154,522,127","680,661,675","2,753,987,998","47,094,865,639","85,917,017,696","133,011,883,336","882,702,922,566","749,691,039,230"
7,"16,222,827","1,092,349,731,112","9,043,469","98,828,370,778","7,172,046","132,250,968,817","139,442,917,664","723,997,183,246","104,415,929,974","1,018,381,692,957",...,"60,125,018","104,476,054,992","11,072,525,677","1,012,981,743","1,298,281,698","93,118,229,361","124,635,761,651","217,753,991,012","1,204,302,332,176","986,548,341,164"
8,"16,221,896","1,655,218,919,934","5,830,283","69,495,223,098","10,390,784","223,921,661,490","164,449,691,635","1,198,406,293,993","187,012,834,871","1,535,575,163,967",...,"293,968,886","187,306,803,758","14,524,350,223","1,615,376,719","1,548,159,725","172,849,670,528","201,791,432,199","374,641,102,727","1,780,169,573,807","1,405,528,471,079"
9,"16,223,470","5,010,404,019,291","1,996,496","24,063,641,503","14,214,627","596,219,211,775","156,981,070,943","4,239,681,834,066","1,061,876,168,463","4,769,377,025,799",...,"38,841,090,217","1,100,717,258,680","29,120,331,277","30,368,805,793","923,418,383","1,101,042,314,813","357,073,176,859","1,458,115,491,672","5,288,418,667,894","3,830,303,176,222"


In [8]:
new_table, _ = calc_new.distribution_tables(None)
print ('New PUF')
new_table

ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'

Exception ignored in: 'pandas._libs.lib.is_bool_array'
ValueError: Buffer dtype mismatch, expected 'Python object' but got 'long'


New PUF


,s006,c00100,num_returns_StandardDed,standard,num_returns_ItemDed,c04470,c04600,c04800,taxbc,c62100,...,c09600,c05800,c07100,othertaxes,refund,iitax,payrolltax,combined,expanded_income,aftertax_income
0,"16,206,016","-43,956,302,370","11,504,053","99,558,547,058","1,536","204,957,779","65,845,928,547","940,693,088","69,544,365","-44,073,202,632",...,0,"69,544,365","3,011,340","94,705,288","1,786,025,251","-1,624,786,938","5,800,890,667","4,176,103,728","-34,253,632,852","-38,429,736,580"
1,"16,206,082","110,475,115,754","13,953,329","120,466,804,131","69,673","726,505,698","84,185,707,320","6,057,146,850","542,655,929","109,964,100,731",...,"42,476,287","585,132,216","7,242,826","61,020,742","14,329,353,888","-13,690,443,755","15,185,214,085","1,494,770,330","151,061,539,045","149,566,768,715"
2,"16,207,411","177,938,895,038","13,965,760","128,903,218,920","300,854","3,480,580,134","108,324,244,188","28,421,497,581","2,754,114,052","175,712,028,474",...,"107,248,365","2,861,362,417","161,857,139","103,523,914","27,281,381,294","-24,478,352,102","24,819,963,713","341,611,611","253,839,351,673","253,497,740,062"
3,"16,205,983","260,974,020,573","14,495,043","134,414,959,135","602,716","7,625,541,332","115,347,776,301","71,306,656,969","7,561,261,806","256,620,179,262",...,"139,514,724","7,700,776,530","986,197,407","190,203,222","23,793,063,438","-16,888,281,094","35,395,225,223","18,506,944,129","370,126,046,701","351,619,102,572"
4,"16,206,852","383,416,891,999","14,339,169","132,920,501,183","1,488,012","20,397,983,616","119,821,577,535","151,358,793,694","17,528,653,994","371,278,355,176",...,"164,342,548","17,692,996,543","2,814,517,004","275,321,807","16,814,732,323","-1,660,930,978","49,669,886,779","48,008,955,801","507,542,397,581","459,533,441,780"
5,"16,206,726","543,216,405,423","13,255,894","129,385,689,681","2,839,470","41,289,171,018","123,582,779,468","274,360,249,638","32,804,989,204","519,510,104,941",...,"212,018,428","33,017,007,631","4,729,112,869","386,082,573","8,087,490,248","20,586,487,088","66,207,845,676","86,794,332,764","678,569,738,754","591,775,405,991"
6,"16,206,520","769,490,943,531","11,599,036","120,363,140,054","4,580,958","74,109,722,143","129,052,082,290","454,057,157,498","58,706,488,575","726,596,668,946",...,"207,864,821","58,914,353,396","6,564,212,510","535,854,229","1,782,348,528","51,103,646,586","89,531,305,607","140,634,952,193","899,044,021,347","758,409,069,154"
7,"16,206,641","1,113,289,780,303","9,157,945","102,893,206,813","7,043,770","129,140,943,884","143,959,240,454","739,557,143,067","105,702,181,000","1,041,321,468,771",...,"262,207,274","105,964,388,274","8,527,803,026","947,011,218","162,965,033","98,220,631,433","126,602,197,007","224,822,828,441","1,224,227,735,991","999,404,907,550"
8,"16,206,668","1,652,891,333,712","6,035,383","72,889,030,624","10,168,296","218,059,961,963","167,560,279,555","1,197,584,387,220","184,404,201,509","1,536,444,773,488",...,"366,544,527","184,770,746,036","10,678,995,150","1,380,263,145","27,646,350","175,444,367,682","198,244,262,772","373,688,630,454","1,779,723,054,240","1,406,034,423,786"
9,"16,206,669","4,964,247,376,357","2,094,666","26,124,739,832","14,101,383","584,920,024,535","160,159,228,649","4,199,372,150,059","1,035,239,493,695","4,749,046,200,455",...,"40,456,938,747","1,075,696,432,443","22,509,980,113","29,743,740,573","1,898,431","1,082,928,294,471","347,296,832,484","1,430,225,126,956","5,276,963,657,561","3,846,738,530,605"


# Diagnostic Table Comparison

In [9]:
curr_diag = calc.diagnostic_table(1)
curr_diag

,2014
Returns (#m),162.2
AGI ($b),"9,811.7"
Itemizers (#m),42.0
Itemized Deduction ($b),"1,109.3"
Standard Deduction Filers (#m),107.8
Standard Deduction ($b),925.9
Personal Exemption ($b),"1,115.8"
Taxable Income ($b),"7,096.7"
Regular Tax ($b),"1,468.5"
AMT Income ($b),"9,276.1"


In [10]:
new_diag = calc_new.diagnostic_table(1)
new_diag

,2014
Returns (#m),162.1
AGI ($b),"9,932.0"
Itemizers (#m),41.2
Itemized Deduction ($b),"1,080.0"
Standard Deduction Filers (#m),110.4
Standard Deduction ($b),979.3
Personal Exemption ($b),"1,159.1"
Taxable Income ($b),"7,123.0"
Regular Tax ($b),"1,445.3"
AMT Income ($b),"9,442.4"


In [11]:
diag_data = pd.DataFrame()
diag_data['SOI'] = soi_stats['Value']
diag_data['New'] = new_diag[2014]
diag_data['Current'] = curr_diag[2014]
diag_data['% Change'] = ((new_diag[2014] / curr_diag[2014]) - 1) * 100
diag_data['New - SOI'] = new_diag[2014] - diag_data['SOI']
diag_data['Current - SOI'] = curr_diag[2014] - diag_data['SOI']
diag_data

,SOI,New,Current,% Change,New - SOI,Current - SOI
Returns (#m),148.6,162.1,162.2,-0.1,13.5,13.6
AGI ($b),"9,771.0","9,932.0","9,811.7",1.2,161.0,40.7
Itemizers (#m),44.0,41.2,42.0,-1.9,-2.8,-2.0
Itemized Deduction ($b),"1,206.7","1,080.0","1,109.3",-2.6,-126.7,-97.4
Standard Deduction Filers (#m),117.4,110.4,107.8,2.4,-7.0,-9.6
Standard Deduction ($b),876.2,979.3,925.9,5.8,103.1,49.7
Personal Exemption ($b),"1,121.6","1,159.1","1,115.8",3.9,37.5,-5.8
Taxable Income ($b),"6,997.9","7,123.0","7,096.7",0.4,125.1,98.8
Regular Tax ($b),nan,"1,445.3","1,468.5",-1.6,nan,nan
AMT Income ($b),nan,"9,442.4","9,276.1",1.8,nan,nan


# Income Levels

In [12]:
def pct_diff(current, new):
    return ((new / current) - 1) * 100

In [13]:
inc_dict = OrderedDict()
inc_dict['New'] = []
inc_dict['Current'] = []
inc_dict['SOI'] = []
inc_dict['Pct Diff'] = []
inc_dict['New - SOI'] = []
inc_dict['Current - SOI'] = []
inc_vars = ['e00200', 'e00300', 'e00600', 'e00650', 'e00900']
soi_inc_vars = ['WAS', 'INT', 'ODIV', 'QDIV', 'BIZ']
for tc, soi in zip(inc_vars, soi_inc_vars):
    new_val = calc_new.weighted_total(tc)
    curr_val = calc.weighted_total(tc)
    soi_val = soi_income[soi].sum()
    inc_dict['New'].append(new_val)
    inc_dict['Current'].append(curr_val)
    inc_dict['SOI'].append(soi_val)
    inc_dict['Pct Diff'].append(pct_diff(curr_val, new_val))
    inc_dict['New - SOI'].append(new_val - soi_val)
    inc_dict['Current - SOI'].append(curr_val - soi_val)
# create data frame to display totals better
inc_df = pd.DataFrame.from_dict(inc_dict)
inc_df.index = ['WAS', 'Taxable Interest', 'Ordinary Dividends', 'Qualified Dividends', 'Business Income']
print ("'Pct Diff' represents the percentage change between the current and new sums")
inc_df

'Pct Diff' represents the percentage change between the current and new sums


,New,Current,SOI,Pct Diff,New - SOI,Current - SOI
WAS,"6,784,950,833,499.0","6,784,952,125,431.1","6,785,880,966,000.0",-0.0,"-930,132,501.0","-928,840,568.9"
Taxable Interest,"90,425,786,010.6","93,894,174,624.3","93,894,281,000.0",-3.7,"-3,468,494,989.4","-106,375.7"
Ordinary Dividends,"254,696,960,387.0","254,702,137,258.3","254,702,232,000.0",-0.0,"-5,271,613.0","-94,741.7"
Qualified Dividends,"180,830,300,363.9","186,804,634,087.7","192,447,708,000.0",-3.2,"-11,617,407,636.1","-5,643,073,912.3"
Business Income,"317,260,010,213.3","317,258,657,428.4","317,248,783,000.0",0.0,"11,227,213.3","9,874,428.4"


# Distribution of Income Variables

In [14]:
new_dist_dict = {}
cur_dist_dict = {}
for item in inc_vars:
    new_dist_dict[item] = distribution(calc_new.array(item), calc_new.array('s006'), calc_new.array('c00100'))
    cur_dist_dict[item] = distribution(calc.array(item), calc.array('s006'), calc.array('c00100'))
new_dist = pd.DataFrame()
cur_dist = pd.DataFrame()
# Add all of the variables to a data frame
for tc, x in zip(inc_vars, soi_inc_vars):
    new_dist[x] = new_dist_dict[tc][1]
    cur_dist[x] = cur_dist_dict[tc][1]
new_dist['AGI Bin'] = index_list()
new_dist['label'] = 'New'
cur_dist['AGI Bin'] = index_list()
cur_dist['label'] = 'Current'
# SOI distribution
items_tups = [('WAS', 'WAS'), ('INT', 'Interest Income'), ('ODIV', 'Ordinary Dividends'),
              ('QDIV', 'Qualified Dividends'), ('BIZ', 'Business Income')]
soi_dist = pd.DataFrame()
for item in items_tups:
    soi_dist[item[0]] = (soi_income[item[0]] / soi_income[item[0]].sum())
soi_dist['AGI Bin'] = index_list()
soi_dist['label'] = 'SOI'

scatter_new = ColumnDataSource(new_dist)
scatter_cur = ColumnDataSource(cur_dist)
scatter_soi = ColumnDataSource(soi_dist)

figure_list = []
for item in items_tups:
    title = 'Percent of Total {} by AGI Bin'.format(item[1])
    f = figure(title=title, x_range=index_list())
    f.xaxis.major_label_orientation = 45
    f.circle(x='AGI Bin', y=item[0], color='green', size=10, legend='New', alpha=0.5,
             source=scatter_new)
    f.circle(x='AGI Bin', y=item[0], color='red', size=10, legend='Current', alpha=0.5,
             source=scatter_cur)
    f.circle(x='AGI Bin', y=item[0], color='blue', size=10, legend='SOI', alpha=0.5,
             source=scatter_soi)
    f.legend.location = 'top_left'
    f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
    hover = HoverTool(tooltips=[('Pct', '@{}%'.format(item[0]))])
    f.add_tools(hover)
    figure_list.append(f)
show(column(figure_list))

In [15]:
new_tot = pd.DataFrame()
cur_tot = pd.DataFrame()
# Add all of the variables to a data frame
for tc, x in zip(inc_vars, soi_inc_vars):
    new_tot[x] = new_dist_dict[tc][0]
    cur_tot[x] = cur_dist_dict[tc][0]
new_tot['AGI Bin'] = index_list()
new_tot['label'] = 'New'
cur_tot['AGI Bin'] = index_list()
cur_tot['label'] = 'Current'

soi_income['AGI Bin'] = index_list()
soi_income['label'] = 'SOI'

total_new = ColumnDataSource(new_tot)
total_cur = ColumnDataSource(cur_tot)
total_soi = ColumnDataSource(soi_income)

figure_list = []
for item in items_tups:
    title = 'Total {} by AGI Bin'.format(item[1])
    f = figure(title=title, x_range=index_list())
    f.xaxis.major_label_orientation = 45
    f.circle(x='AGI Bin', y=item[0], color='green', size=10, legend='New', alpha=0.5,
             source=total_new)
    f.circle(x='AGI Bin', y=item[0], color='red', size=10, legend='Current', alpha=0.5,
             source=total_cur)
    f.circle(x='AGI Bin', y=item[0], color='blue', size=10, legend='SOI', alpha=0.5,
             source=total_soi)
    f.legend.location = 'top_left'
    f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
    hover = HoverTool(tooltips=[('Pct', '@{}%'.format(item[0]))])
    f.add_tools(hover)
    figure_list.append(f)
show(column(figure_list))

# Itemized Deduction Amounts

In [16]:
deductions_new = {}
deductions_cur = {}
deduction_list = ['Medical Expenses', 'State and Local Taxes', 'Real Estate Taxes',
                  'Interest Paid', 'Charitable Cash Contributions',
                  'Charitable Non-Cash Contributions', 'Total Misc. Expenses',
                  'Net Casualty or Loss']
deduction_vars = ['e17500', 'e18400', 'e18500', 'e19200', 'e19800', 'e20100',
                  'e20400', 'g20500']
for ded, var in zip(deduction_list, deduction_vars):
    deductions_new[ded] = (new_df[var][new_df['c04470'] > 0] *
                           new_df['s006'][new_df['c04470'] > 0]).sum()
    deductions_cur[ded] = (calc_df[var][calc_df['c04470'] > 0] *
                           calc_df['s006'][calc_df['c04470'] > 0]).sum()
ded_new_df = pd.DataFrame.from_dict(deductions_new, 'index')
ded_new_df.columns = ['Total']
ded_new_df['source'] = 'New'
ded_cur_df = pd.DataFrame.from_dict(deductions_cur, 'index')
ded_cur_df.columns = ['Total']
ded_cur_df['source'] = 'Current'

soi_deductions['source'] = 'SOI'

ded_cds_new = ColumnDataSource(ded_new_df)
ded_cds_new.add(data=[i - .5 for i in range(1, 9)], name='xaxis')
ded_cds_cur = ColumnDataSource(ded_cur_df)
ded_cds_cur.add(data=[i - .25 for i in range(1, 9)], name='xaxis')
ded_cds_soi = ColumnDataSource(soi_deductions)
ded_cds_soi.add(data=[i - .75 for i in range(1, 9)], name='xaxis')

'xaxis'

In [17]:
xaxis_list = ['Interest Paid', 'Total Misc. Expenses', 'Medical Expenses',
              'Charitable Non-Cash Contributions', 'Real Estate Taxes',
              'State and Local Taxes', 'Charitable Cash Contributions',
              'Net Casualty or Loss']
f = figure(title='Itemized Deduction Totals', x_range=xaxis_list, width=800)
#f = figure(title='Itemized Deduction Totals')
f.xaxis.major_label_orientation = 45
f.vbar(x='xaxis', bottom=0, top='Total', legend='New',
       color='green', alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_new)
f.vbar(x='xaxis', bottom=0, top='Total', legend='Current', color='red',
       alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_cur)
f.vbar(x='xaxis', bottom=0, top='Total', legend='SOI', color='blue',
       alpha=0.5, line_alpha=1,
       width=0.25, source=ded_cds_soi)
hover = HoverTool(tooltips=[('Total', '@Total{$0.00a}')])
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
f.add_tools(hover)

show(f)

In [18]:
ded_error_df = pd.DataFrame()
ded_error_df['Difference: New'] = ded_new_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: New'] = 100 * ded_error_df['Difference: New'] / soi_deductions['Total']
ded_error_df['Difference: Current'] = ded_cur_df['Total'] - soi_deductions['Total']
ded_error_df['% Difference: Current'] = 100 * ded_error_df['Difference: Current'] / soi_deductions['Total']
print ('Error in Itemized Deductions Relative to SOI Totals')
ded_error_df

Error in Itemized Deductions Relative to SOI Totals


,Difference: New,% Difference: New,Difference: Current,% Difference: Current
Medical Expenses,"-12,054,421,348.8",-9.3,"-10,263,322,756.7",-8.0
State and Local Taxes,"-18,630,489,216.9",-5.7,"-15,964,512,279.0",-4.9
Real Estate Taxes,"2,384,469,754.7",1.3,"17,294,916,696.2",9.6
Interest Paid,"-16,328,258,245.9",-5.3,"-7,293,196,129.2",-2.4
Charitable Cash Contributions,"-6,384,396,376.2",-4.1,"-4,690,078,031.9",-3.0
Charitable Non-Cash Contributions,"-27,175,410,720.7",-41.6,"-31,545,208,332.2",-48.3
Total Misc. Expenses,"3,753,519,968.3",3.0,"4,144,485,540.8",3.3
Net Casualty or Loss,"2,863,801,247.1",129.9,"2,271,378,406.5",103.0


### Itemizing Rate

In [19]:
calc_df['itemizer'] = np.where(calc_df['c04470'] > 0, 1, 0)
new_df['itemizer'] = np.where(new_df['c04470'] > 0, 1, 0)
pr_new = percentile(new_df, 'itemizer', 100, 'e00200', 's006')
pr_cur = percentile(calc_df, 'itemizer', 100, 'e00200', 's006')
item_index = pr_new.index
pr_cds_new = ColumnDataSource({'rate': pr_new,
                               'index': item_index})
pr_cds_cur = ColumnDataSource({'rate': pr_cur,
                                'index': item_index})

hover_ded = HoverTool(tooltips=[('Percentile', '@index'), ('Itemizing Rate', '@rate{0.00%}')])
f = figure(title='Itemizing Rate by Wage Percentile',
           x_axis_label='Wage Percentile')
f.add_tools(hover_ded)
f.line('index', 'rate', legend='New', source=pr_cds_new)
f.line('index', 'rate', legend='Current', color='red', source=pr_cds_cur)
f.yaxis[0].formatter = NumeralTickFormatter(format='0.00%')
f.legend.location = 'top_left'
show(f)

In [20]:
itemizers_new = copy.deepcopy(new_df[new_df['c04470'] > 0])
mean_new = percentile(itemizers_new, 'c04470', 100, 'e00200', 's006')
itemizers = copy.deepcopy(calc_df[calc_df['c04470'] > 0])
mean_cur = percentile(itemizers, 'c04470', 100, 'e00200', 's006')
mean_cds_new = ColumnDataSource({'mean': mean_new,
                                 'xaxis': item_index})
mean_cds_cur = ColumnDataSource({'mean': mean_cur,
                                 'xaxis': item_index})

hover = HoverTool(tooltips=[('Percentile', '@xaxis'), ('Mean', '@mean{$0.00a}')])
f = figure(title='Mean Itemized Deduction Total Among Itemizers',
           x_axis_label='Wage Percentile')
f.add_tools(hover)
f.line('xaxis', 'mean', legend='New', source=mean_cds_new)
f.line('xaxis', 'mean', legend='Current', color='red', source=mean_cds_cur)
f.legend.location = 'top_left'
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')
show(f)

# Tax Liability by Year

In [21]:
# new liabilities
li_new = {'Income': [calc_new.weighted_total('iitax')],
          'Payroll': [calc_new.weighted_total('payrolltax')],
          'Combined': [calc_new.weighted_total('combined')],
          'source': ['New']}
# current liabilities
li_cur = {'Income': [calc.weighted_total('iitax')],
          'Payroll': [calc.weighted_total('payrolltax')],
          'Combined': [calc.weighted_total('combined')],
          'source': ['Current']}
years = [2014]
for year in range(2015, 2028):
    years.append(year)
    calc.advance_to_year(year)
    calc.calc_all()
    calc_new.advance_to_year(year)
    calc_new.calc_all()
    li_new['Income'].append(calc_new.weighted_total('iitax'))
    li_new['Payroll'].append(calc_new.weighted_total('payrolltax'))
    li_new['Combined'].append(calc_new.weighted_total('combined'))
    li_new['source'].append('New')
    li_cur['Income'].append(calc.weighted_total('iitax'))
    li_cur['Payroll'].append(calc.weighted_total('payrolltax'))
    li_cur['Combined'].append(calc.weighted_total('combined'))
    li_cur['source'].append('Current')
li_new['years'] = years
li_cur['years'] = years

In [22]:
# ColumnDataSources for plotting
li_cds_new = ColumnDataSource(li_new)
li_cds_cur = ColumnDataSource(li_cur)

In [29]:
f = figure(title='Tax Liabilities 2015-2027')
f.width = 800
f.height = 500
f.line('years', 'Income', color='blue', line_width=2, legend='Income-Current',
       source=li_cds_cur)
f.line('years', 'Income', color='cyan', line_width=2, legend='Income-New',
       source=li_cds_new)
f.line('years', 'Payroll', color='purple', line_width=2, legend='Payroll-Current',
       source=li_cds_cur)
f.line('years', 'Payroll', color='magenta', line_width=2, legend='Payroll-New',
       source=li_cds_new)
f.line('years', 'Combined', color='green', line_width=2, legend='Combined-Current',
       source=li_cds_cur)
f.line('years', 'Combined', color='lime', line_width=2, legend='Combined-New',
       source=li_cds_new)

c1 = f.circle('years', 'Income', color='blue', size=5, legend='Income-Current',
              source=li_cds_cur)
c2 = f.circle('years', 'Income', color='cyan', size=5, legend='Income-New',
              source=li_cds_new)
c3 = f.circle('years', 'Payroll', color='purple', size=5, legend='Payroll-Current',
              source=li_cds_cur)
c4 = f.circle('years', 'Payroll', color='magenta', size=5, legend='Payroll-New',
              source=li_cds_new)
c5 = f.circle('years', 'Combined', color='green', size=5, legend='Combined-Current',
              source=li_cds_cur)
c6 = f.circle('years', 'Combined', color='lime', size=5, legend='Combined-New',
              source=li_cds_new)
f.legend.location = 'top_left'
f.xaxis[0].ticker.desired_num_ticks = 12
f.xaxis.minor_tick_line_color = None
f.yaxis[0].formatter = NumeralTickFormatter(format='$0.00a')

hover = HoverTool(tooltips=[('PUF', '@source'), ('Year', '@years'),
                            ('Income', '@Income{$0.00a}')],
                  renderers=[c1, c2])
hover2 = HoverTool(tooltips=[('PUF', '@source'), ('Year', '@years'),
                             ('Payroll', '@Payroll{$0.00a}')],
                  renderers=[c3, c4])
hover3 =HoverTool(tooltips=[('PUF', '@source'), ('Year', '@years'),
                            ('Combined', '@Combined{$0.00a}')],
                  renderers=[c5, c6])
f.add_tools(hover, hover2, hover3)
show(f)

In [24]:
ldf_current = pd.DataFrame(li_cur, index=years)
ldf_current.drop(['source', 'years'], axis=1, inplace=True)
print ('Current Tax Liabilities by Year')
ldf_current

Current Tax Liabilities by Year


,Combined,Income,Payroll
2014,"2,324,811,726,624.2","1,371,011,564,057.3","953,800,162,566.9"
2015,"2,496,405,141,206.3","1,499,687,317,297.4","996,717,823,908.9"
2016,"2,572,374,238,835.4","1,539,019,632,733.9","1,033,354,606,101.5"
2017,"2,725,712,039,147.9","1,637,704,761,092.9","1,088,007,278,055.0"
2018,"2,667,916,509,164.4","1,530,852,268,398.2","1,137,064,240,766.3"
2019,"2,773,772,497,820.2","1,591,742,617,388.6","1,182,029,880,431.7"
2020,"2,878,645,055,893.6","1,654,554,132,252.4","1,224,090,923,641.2"
2021,"2,994,094,216,895.6","1,725,484,401,397.6","1,268,609,815,498.0"
2022,"3,122,359,171,982.6","1,805,000,695,989.7","1,317,358,475,993.0"
2023,"3,262,271,239,700.0","1,892,909,990,596.5","1,369,361,249,103.5"


In [25]:
ldf_new = pd.DataFrame(li_new, index=years)
ldf_new.drop(['source', 'years'], axis=1, inplace=True)
print ('New Tax Liabilities by Year')
ldf_new

New Tax Liabilities by Year


,Combined,Income,Payroll
2014,"2,328,694,256,406.3","1,369,940,632,394.1","958,753,624,012.2"
2015,"2,514,619,881,589.3","1,503,722,884,185.4","1,010,896,997,404.0"
2016,"2,424,699,952,421.1","1,369,705,592,017.0","1,054,994,360,404.1"
2017,"2,558,779,503,957.4","1,455,120,339,554.9","1,103,659,164,402.5"
2018,"2,678,866,195,380.7","1,525,891,648,314.6","1,152,974,547,066.1"
2019,"2,783,880,995,734.4","1,585,742,322,844.7","1,198,138,672,889.7"
2020,"2,887,506,598,147.4","1,647,009,960,641.0","1,240,496,637,506.4"
2021,"3,003,659,592,504.4","1,717,999,668,169.9","1,285,659,924,334.4"
2022,"3,133,575,600,428.0","1,798,501,044,332.5","1,335,074,556,095.6"
2023,"3,273,406,485,430.8","1,885,696,118,196.9","1,387,710,367,233.9"


In [26]:
print ('Difference in Tax Liabilities by Year')
diff_df = ldf_new - ldf_current
diff_df = diff_df.append(diff_df.sum(), ignore_index=True)
years.append('Total')
diff_df.index = years
diff_df

Difference in Tax Liabilities by Year


,Combined,Income,Payroll
2014,"3,882,529,782.1","-1,070,931,663.2","4,953,461,445.3"
2015,"18,214,740,383.0","4,035,566,888.0","14,179,173,495.1"
2016,"-147,674,286,414.3","-169,314,040,717.0","21,639,754,302.7"
2017,"-166,932,535,190.5","-182,584,421,538.0","15,651,886,347.5"
2018,"10,949,686,216.3","-4,960,620,083.5","15,910,306,299.8"
2019,"10,108,497,914.2","-6,000,294,543.8","16,108,792,458.0"
2020,"8,861,542,253.8","-7,544,171,611.4","16,405,713,865.2"
2021,"9,565,375,608.8","-7,484,733,227.7","17,050,108,836.4"
2022,"11,216,428,445.4","-6,499,651,657.2","17,716,080,102.6"
2023,"11,135,245,730.8","-7,213,872,399.7","18,349,118,130.4"


In [27]:
print ('Pct. Change in Tax Liabilities by Year')
((ldf_new - ldf_current) / ldf_current) * 100

Pct. Change in Tax Liabilities by Year


,Combined,Income,Payroll
2014,0.2,-0.1,0.5
2015,0.7,0.3,1.4
2016,-5.7,-11.0,2.1
2017,-6.1,-11.1,1.4
2018,0.4,-0.3,1.4
2019,0.4,-0.4,1.4
2020,0.3,-0.5,1.3
2021,0.3,-0.4,1.3
2022,0.4,-0.4,1.3
2023,0.3,-0.4,1.3
